In [ ]:
import pandas as pd
from glob import glob
import os
import numpy as np
import pandas as pd
import random
from statsmodels.stats import inter_rater as irr


In [ ]:
def load_anno(data_dir: str = "anno/", verbose: bool = False) -> pd.DataFrame:
    dfs = []
    c = 0
    for file in glob(data_dir + "*.csv"):
        if verbose:
            print(f"Reading in:   {file}")
        test = pd.read_csv(file)
        name = file.split("/")[1].split(".")[0]
        test["name"] = name
        dfs.append(test)
        c += 1

    if verbose:
        print(f"Loaded {c} files")

    concat_df = pd.concat(dfs)
    return concat_df.reset_index().iloc[:, 1:]

In [ ]:
anno = load_anno()
anno["sentiment"] = anno.label.str.split("#")
anno["sentiment"].dropna(inplace=True)
anno[["anno_rating", "sentiment"]] = anno.sentiment.apply(pd.Series)

# Rating
IRR fleiss kappa score of annotated rating between annotators

In [ ]:
oliver = anno[anno.name=='oliver'].anno_rating
bertram = anno[anno.name=='bertram'].anno_rating
bjarke = anno[anno.name=='bjarke'].anno_rating
sebbas = anno[anno.name=='sebbas'].anno_rating
max = anno[anno.name=='max'].anno_rating
ratings = list(zip(oliver, bertram, sebbas, bjarke, max))
dats, cats = irr.aggregate_raters(ratings)
print(irr.fleiss_kappa(dats, method='fleiss'))

# Sentiment
IRR fleiss kappa score of annotated sentiment between annotators

In [ ]:
oliver = anno[anno.name=='oliver'].sentiment
bertram = anno[anno.name=='bertram'].sentiment
bjarke = anno[anno.name=='bjarke'].sentiment
sebbas = anno[anno.name=='sebbas'].sentiment
max = anno[anno.name=='max'].sentiment
sentiment_rating = list(zip(oliver, bertram, sebbas, bjarke, max))
dats, cats = irr.aggregate_raters(sentiment_rating)
print(irr.fleiss_kappa(dats, method='fleiss'))

In [ ]:
import pandas as pd
from glob import glob
import os
import numpy as np
import pandas as pd
import random
review_df = pd.read_feather("reviews_dataset.feather")

def load_anno(data_dir: str = "anno/", verbose: bool = False) -> pd.DataFrame:
    dfs = []
    c = 0
    for file in glob(data_dir + "*.csv"):
        if verbose:
            print(f"Reading in:   {file}")
        test = pd.read_csv(file)
        name = file.split("/")[1].split(".")[0]
        test["name"] = name
        dfs.append(test)
        c += 1

    if verbose: 
        print(f"Loaded {c} files")

    concat_df = pd.concat(dfs)
    return concat_df.reset_index().iloc[:, 1:]

anno = load_anno()


annotation = load_anno()
annotation["sentiment"] = annotation.label.str.split("#")
annotation[["anno_rating", "sentiment"]] = annotation.sentiment.apply(pd.Series)
annotation["anno_rating"] = annotation.anno_rating.apply(int)
review_df["model_sentiment"] = review_df["sentiment"]
review_df.drop(columns=["sentiment"], inplace=True)
review_df["advice"].fillna("", inplace=True)
review_df["text"] = "Title: \n" + review_df["review_title"] + "\n Pros: \n" + review_df["pros"] + "\n Cons: \n" + review_df["cons"] + "\n Advice: \n" + review_df["advice"]
review_df["text"] = [txt.replace("\r", "") for txt in review_df.text]

merged_tmp = annotation.merge(review_df, on=["text"]).drop_duplicates()

agg_senti = merged_tmp.groupby('text', as_index=False)['sentiment'].agg(pd.Series.mode)
agg_senti["agg_sentiment"] = [x[0] if isinstance(x, np.ndarray) else x for x in agg_senti.sentiment]

agg_anno = merged_tmp.groupby('text', as_index=False).mean('anno_rating')
agg_anno['agg_anno'] = [round(rating) for rating in agg_anno.anno_rating]

merged = merged_tmp.merge(agg_anno[['text', 'agg_anno']], on=["text"]).drop_duplicates()
merged = merged.merge(agg_senti[['text', 'agg_sentiment']], on=["text"]).drop_duplicates()

In [ ]:
tmp = merged.groupby("text", as_index=False).first()

In [ ]:
tmp['int_anno_sent'] = [0 if x == "negative" else 1 if x == "neutral" else 2 for x in tmp.agg_sentiment]
tmp['int_model_sent'] = [0 if x == "negative" else 1 if x == "neutral" else 2 for x in tmp.model_sentiment]

# Aggregated Rating
Aggregated annotator rating vs review-score.

In [ ]:
anno_rating = tmp.agg_anno.apply(int)
rating = tmp.rating.apply(int)

from sklearn.metrics import cohen_kappa_score

print(cohen_kappa_score(anno_rating, rating))


# Aggregated Sentiment
Aggregated annotator sentiment vs model sentiment

In [ ]:
anno_sent = tmp.int_anno_sent.apply(int)
model_sent = tmp.int_model_sent.apply(int)

from sklearn.metrics import cohen_kappa_score

print(cohen_kappa_score(anno_sent, model_sent))